In [40]:
%pip install nltk pandas stanza scikit-learn geopy folium



   -------- ------------------------------- 1/5 [geographiclib]
   ---------------- ----------------------- 2/5 [geopy]
   ---------------- ----------------------- 2/5 [geopy]
   ------------------------ --------------- 3/5 [branca]
   -------------------------------- ------- 4/5 [folium]
   -------------------------------- ------- 4/5 [folium]
   -------------------------------- ------- 4/5 [folium]
   ---------------------------------------- 5/5 [folium]

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import stanza
import re
import requests
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Inicializácia Stanza pre češtinu (na lemmatizáciu a POS tagging)
stanza.download('cs')  # Stiahnutie českého modelu
nlp = stanza.Pipeline(lang='cs', processors='tokenize,lemma,pos')

# Stiahnutie NLTK dát
nltk.download('punkt')

# Načítanie CSV súboru
# Predpokladáme, že CSV má stĺpec 'text' s českými textovými dátami
df = pd.read_csv('user1.csv')  # Nahraď 'data.csv' správnym názvom súboru
print("Prvé riadky dát:")
print(df.head())

# Základné štatistiky
print("\nZákladné informácie o dátach:")
print(df.info())

2025-05-15 18:00:50 INFO: Downloaded file to C:\Users\vladimir\stanza_resources\resources.json
2025-05-15 18:00:50 INFO: Downloading default packages for language: cs (Czech) ...
2025-05-15 18:00:50 INFO: File exists: C:\Users\vladimir\stanza_resources\cs\default.zip
2025-05-15 18:00:51 INFO: Finished downloading models and saved to C:\Users\vladimir\stanza_resources
2025-05-15 18:00:51 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-05-15 18:00:51 INFO: Downloaded file to C:\Users\vladimir\stanza_resources\resources.json
2025-05-15 18:00:51 WARNING: Language cs package default expects mwt, which has been added
2025-05-15 18:00:52 INFO: Loading these models for language: cs (Czech):
| Processor | Package      |
----------------------------
| tokenize  | pdt          |
| mwt       | pdt          |
| pos       | pdt_nocharlm |
| lemma     

Prvé riadky dát:
   Hviezdičky                                             Miesto  \
0           5                                   DinoPark Liberec   
1           5                             Doučovaní Doma Liberec   
2           5  Liberecká výšina - rozhledna s hotelem a resta...   
3           5                                          Wroclavia   
4           5                           Stadion Tarczyński Arena   

                                       Adresa  \
0              Palachova 1404, 460 90 Liberec   
1          Krátká 388/2, 460 07 Liberec-Jeřáb   
2  Wolkerova 251, 460 15 Liberec-Starý Harcov   
3             Sucha 1, 50-086 Wrocław, Polsko   
4        al. Śląska 1, 54-118 Wrocław, Polsko   

                                                Text       Datum  
0  Stále atraktivní místo pro rodiče s dětmi. Vst...  2025-05-09  
1  Velmi příjemné prostředí a podmínky pro výuku....  2025-05-15  
2  Navštívena pouze restaurace a sál. Ranní cater...  2025-05-15  
3  Moderní 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vladimir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
#Lowercase a čistenie textu

def clean_text(text):
    # Prevod na malé písmená
    text = text.lower()
    #odstran bodky a čiarky
    text = re.sub(r'[.,;!?]', '', text)
    text = ' '.join(text.split())
    return text

# Aplikácia čistenia na stĺpec 'text'
df['cleaned_text'] = df['Text'].apply(clean_text)
print("\nUkážka vyčisteného textu:")
print(df[['Text', 'cleaned_text']].head())


Ukážka vyčisteného textu:
                                                Text  \
0  Stále atraktivní místo pro rodiče s dětmi. Vst...   
1  Velmi příjemné prostředí a podmínky pro výuku....   
2  Navštívena pouze restaurace a sál. Ranní cater...   
3  Moderní obchodní centrum včetně Parkování.  Po...   
4  Dojezd ke stadionu na jedničku. Velká parkovac...   

                                        cleaned_text  
0  stále atraktivní místo pro rodiče s dětmi vstu...  
1  velmi příjemné prostředí a podmínky pro výuku ...  
2  navštívena pouze restaurace a sál ranní cateri...  
3  moderní obchodní centrum včetně parkování poho...  
4  dojezd ke stadionu na jedničku velká parkovací...  


In [13]:
def tokenize_text(text):
    doc = nlp(text)
    return [word.text for sent in doc.sentences for word in sent.words]

# Aplikácia tokenizácie
df['tokens'] = df['cleaned_text'].apply(tokenize_text)
print("\nUkážka tokenizovaného textu:")
print(df[['cleaned_text', 'tokens']].head())


Ukážka tokenizovaného textu:
                                        cleaned_text  \
0  stále atraktivní místo pro rodiče s dětmi vstu...   
1  velmi příjemné prostředí a podmínky pro výuku ...   
2  navštívena pouze restaurace a sál ranní cateri...   
3  moderní obchodní centrum včetně parkování poho...   
4  dojezd ke stadionu na jedničku velká parkovací...   

                                              tokens  
0  [stále, atraktivní, místo, pro, rodiče, s, dět...  
1  [velmi, příjemné, prostředí, a, podmínky, pro,...  
2  [navštívena, pouze, restaurace, a, sál, ranní,...  
3  [moderní, obchodní, centrum, včetně, parkování...  
4  [dojezd, ke, stadionu, na, jedničku, velká, pa...  


In [14]:

# Načítanie stop slov z GitHubu
# používame české stop slová z repozitára stopwords-iso link
stopwords_url = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-cs/master/stopwords-cs.json'
response = requests.get(stopwords_url)
czech_stop_words = set(json.loads(response.text))

def remove_stopwords(tokens):
    return [token for token in tokens if token not in czech_stop_words]

# Odstránenie stop slov
df['tokens_no_stopwords'] = df['tokens'].apply(remove_stopwords)
print("\nUkážka tokenov po odstránení stop slov:")
print(df[['tokens', 'tokens_no_stopwords']].head())


Ukážka tokenov po odstránení stop slov:
                                              tokens  \
0  [stále, atraktivní, místo, pro, rodiče, s, dět...   
1  [velmi, příjemné, prostředí, a, podmínky, pro,...   
2  [navštívena, pouze, restaurace, a, sál, ranní,...   
3  [moderní, obchodní, centrum, včetně, parkování...   
4  [dojezd, ke, stadionu, na, jedničku, velká, pa...   

                                 tokens_no_stopwords  
0  [stále, atraktivní, místo, rodiče, dětmi, vstu...  
1  [velmi, příjemné, prostředí, podmínky, výuku, ...  
2  [navštívena, restaurace, sál, ranní, catering,...  
3  [moderní, obchodní, centrum, včetně, parkování...  
4  [dojezd, stadionu, jedničku, velká, parkovací,...  


In [15]:
#Ceska lematizácia
def lemmatize_text(text):
    doc = nlp(text)
    return [word.lemma for sent in doc.sentences for word in sent.words]

# Aplikácia lemmatizácie
df['lemmatized_tokens'] = df['tokens_no_stopwords'].apply(lambda tokens: lemmatize_text(' '.join(tokens)))
print("\nUkážka lemmatizovaných tokenov:")
print(df[['tokens_no_stopwords', 'lemmatized_tokens']].head())


Ukážka lemmatizovaných tokenov:
                                 tokens_no_stopwords  \
0  [stále, atraktivní, místo, rodiče, dětmi, vstu...   
1  [velmi, příjemné, prostředí, podmínky, výuku, ...   
2  [navštívena, restaurace, sál, ranní, catering,...   
3  [moderní, obchodní, centrum, včetně, parkování...   
4  [dojezd, stadionu, jedničku, velká, parkovací,...   

                                   lemmatized_tokens  
0  [stále, atraktivní, místo, rodič, dítě, vstupn...  
1  [velmi, příjemný, prostředí, podmínka, výuka, ...  
2  [navštívený, restaurace, sál, ranní, catering,...  
3  [moderní, obchodní, centrum, včetně, parkování...  
4  [dojezd, stadion, jednička, velký, parkovací, ...  


In [ ]:
# RUNNING THE CODE CIRCA 20 MINUTES HTML ALREADY GENERATED
#HTML IS NOT COMPATIBLE WITH GITHUBIO SITE SO USE BROWSER TO OPEN IT
# from geopy.geocoders import Nominatim
# from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
# import folium
# import time
# from geopy.extra.rate_limiter import RateLimiter

# def extract_location(address, max_retries=3):
#     geolocator = Nominatim(user_agent="my_app_v1")
#     # Create a rate-limited version of the geocoder
#     geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=max_retries)
    
#     try:
#         location = geocode(address)
#         if location:
#             return (location.latitude, location.longitude)
#         return None
#     except (GeocoderTimedOut, GeocoderUnavailable) as e:
#         print(f"Error geocoding {address}: {str(e)}")
#         return None

# # Create a map centered on Europe
# m = folium.Map(location=[50.0755, 14.4378], zoom_start=4)

# # Extract locations from the address column with progress tracking
# print("Geocoding addresses...")
# for i, address in enumerate(df['Adresa']):
#     print(f"Processing address {i+1}/{len(df['Adresa'])}: {address}")
#     coords = extract_location(address)
#     if coords:
#         folium.Marker(
#             coords,
#             popup=address,
#             icon=folium.Icon(color='red', icon='info-sign')
#         ).add_to(m)
#     else:
#         print(f"Could not geocode address: {address}")

# # Save the map
# output_file = 'locations_map.html'
# m.save(output_file)
# print(f"\nMap has been saved to {output_file}")

Geocoding addresses...
Processing address 1/704: Palachova 1404, 460 90 Liberec
Processing address 2/704: Krátká 388/2, 460 07 Liberec-Jeřáb
Processing address 3/704: Wolkerova 251, 460 15 Liberec-Starý Harcov
Processing address 4/704: Sucha 1, 50-086 Wrocław, Polsko
Processing address 5/704: al. Śląska 1, 54-118 Wrocław, Polsko
Processing address 6/704: Felberova 1, 460 05 Liberec-Kristiánov
Processing address 7/704: Świętej Elżbiety 1/2, 50-111 Wrocław, Polsko
Processing address 8/704: Nová huť 687, 471 14 Kamenický Šenov
Processing address 9/704: Henryka Sienkiewicza 23, 50-335 Wrocław, Polsko
Processing address 10/704: Karlův most, 110 00 Praha 1
Processing address 11/704: Janov nad Nisou 524, 468 11 Janov nad Nisou
Processing address 12/704: Na Sboře 2271, 511 01 Turnov 1
Could not geocode address: Na Sboře 2271, 511 01 Turnov 1
Processing address 13/704: Plynární 167, 170 00 Praha 7-Holešovice
Processing address 14/704: Zámečnická 561/4, 460 01 Liberec
Processing address 15/704: 

RateLimiter caught an error, retrying (0/3 tries). Called with (*('460 14, Lesní 461/41, 460 14 Liberec',), **{}).
Traceback (most recent call last):
  File "c:\Users\vladimir\Desktop\Projects\TM\.venv\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "c:\Users\vladimir\Desktop\Projects\TM\.venv\Lib\site-packages\urllib3\connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
  File "C:\Users\vladimir\AppData\Local\Programs\Python\Python313\Lib\http\client.py", line 1430, in getresponse
    response.begin()
    ~~~~~~~~~~~~~~^^
  File "C:\Users\vladimir\AppData\Local\Programs\Python\Python313\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ~~~~~~~~~~~~~~~~~^^
  File "C:\Users\vladimir\AppData\Local\Programs\Python\Python313\Lib\http\client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8

Could not geocode address: 460 14, Lesní 461/41, 460 14 Liberec
Processing address 508/704: Unnamed Road, 463 12 Liberec
Could not geocode address: Unnamed Road, 463 12 Liberec
Processing address 509/704: Nad Školou 1675, 463 11 Liberec 30-Vratislavice nad Nisou
Could not geocode address: Nad Školou 1675, 463 11 Liberec 30-Vratislavice nad Nisou
Processing address 510/704: Vratislavice nad Nisou, 463 11 Liberec 30
Processing address 511/704: Brněnská 1131, 664 42 Modřice
Processing address 512/704: Sousedská 600, 460 01 Liberec
Processing address 513/704: Bohdanečská 192, 530 02 Pardubice VII
Could not geocode address: Bohdanečská 192, 530 02 Pardubice VII
Processing address 514/704: 5. května 175/7, 460 01 Liberec
Processing address 515/704: Ke Hluboké, 463 12 Liberec
Processing address 516/704: Doubská 997, 460 06 Liberec-Rochlice
Processing address 517/704: Dr. M. Horákové 412/55, 460 07 nečleněné město-Horní Růžodol
Could not geocode address: Dr. M. Horákové 412/55, 460 07 nečleněn

In [ ]:
# Inicializácia TF-IDF vektorizéra ??? Potreba co dalej ?
vectorizer = TfidfVectorizer(max_features=1000)  # Obmedzíme na 1000 najčastejších slov ??
tfidf_matrix = vectorizer.fit_transform(df['lemmatized_tokens'].apply(lambda x: ' '.join(x)))
print("\nUkážka TF-IDF matice:")

# Prevedenie na DataFrame pre lepšiu prehľadnosť
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
print("\nUkážka TF-IDF matice:")
print(tfidf_df.head())


Ukážka TF-IDF matice:

Ukážka TF-IDF matice:
    00   10   11   12   13   14   15  170   18   20  ...  špinava  špinavej  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...      0.0       0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...      0.0       0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...      0.0       0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...      0.0       0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...      0.0       0.0   

   špinavý  štěkat  štěstí     šunka  žampion  žebro  židlička  žádný  
0      0.0     0.0     0.0  0.296158      0.0    0.0       0.0    0.0  
1      0.0     0.0     0.0  0.000000      0.0    0.0       0.0    0.0  
2      0.0     0.0     0.0  0.000000      0.0    0.0       0.0    0.0  
3      0.0     0.0     0.0  0.000000      0.0    0.0       0.0    0.0  
4      0.0     0.0     0.0  0.000000      0.0    0.0       0.0    0.0  

[5 rows x 1000 columns]
